In [1]:
import subprocess

import pandas as pd
import numpy as np
import openpyxl

In [2]:
TARGET_COLUMNS = [
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'alt_hcpcs_cpt',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [3]:
# https://www.piedmont.org/patient-tools/hospital-price-transparency
#ccn = "110083"
#url = "https://www.piedmont.org/media/file/580566213_piedmont-atlanta-hospital_standardcharges.xls"

#ccn = "110074"
#url = "https://www.piedmont.org/media/file/582179986_piedmont-athens-hospital_standardcharges.xls"

#ccn = "110064"
#url = "https://www.piedmont.org/media/file/581685139_piedmont-columbus-regional-midtown-hospital_standardcharges.xls"

ccn = "110229"
url = "https://www.piedmont.org/media/file/205077249_piedmont-newnan-hospital_standardcharges.xls"

filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]
ein

'20-5077249'

In [4]:
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘205077249_piedmont-newnan-hospital_standardcharges.xls’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://www.piedmont.org/media/file/205077249_piedmont-newnan-hospital_standardcharges.xls', '-O', '205077249_piedmont-newnan-hospital_standardcharges.xls'], returncode=1)

In [5]:
df_in = pd.read_excel(filename, skiprows=[1,2,3])
df_in

,Piedmont Newnan Hospital
0,"Procedure,Service,Description,HCPCS Code,Rev C..."
1,"MS579,Inpatient,Other Skin, Subcutaneous Tissu..."
2,"MS653,Inpatient,Major Bladder Procedures With ..."
3,"MS570,Inpatient,Skin Debridement With McC,N/A,..."
4,"MS571,Inpatient,Skin Debridement With Cc,N/A,N..."
...,...
55777,"129456136,Supplies,AUG TIBIAL 10MM MBT 4 KNEE ..."
55778,"129456126,Supplies,AUG TIBIAL 10MM MBT 2.5 KNE..."
55779,"129456125,Supplies,WEDGE STEP REV CEMENT 2.5X5..."
55780,"129456121,Supplies,AUG TIBIAL 10MM MBT 2 KNEE ..."


In [6]:
# HACK!!!
d = df_in.iloc[0].to_dict()
k = list(d.keys())[0]
columns = d[k].split(',')
columns

['Procedure',
 'Service',
 'Description',
 'HCPCS Code',
 'Rev Code',
 'MSDRG',
 'NDC',
 'Charges',
 'Aetna HMO/POS/OA',
 'Ambetter',
 'Managed Medicaid - Amerigroup',
 'Blue Cross HMO',
 'Blue Cross PPO',
 'Managed Medicaid - Caresource',
 'Cigna HMO/POS/OA',
 'Managed Medicare - Cigna Healthspring',
 'Managed Medicare - Clover Health',
 'Cigna Local+',
 'Managed Medicare - Georgia Health Advantage',
 'Humana HMO',
 'Managed Medicare - Humana',
 'Kaiser',
 'Managed Medicare - Kaiser',
 'Multiplan',
 'Managed Medicaid - PeachState',
 'Managed Medicare - Pruitt Health',
 'Managed Medicare - Sonder',
 'PHCS',
 'Managed Medicare - Wellcare',
 'United Healthcare',
 'Self-Pay',
 'Min Fee',
 'Max Fee']

In [7]:
df_in = df_in.iloc[1:] 
records = df_in[k].to_list()
records = list(map(lambda r: r.split(","), records))

# Sometimes description is comma-separated, which shifts the indices by one.
# This little trick fixes that problem.
def fix_record_list(r):
    while len(r) > len(columns):
        r[1] = r[1] + "," + r[2]
        del r[2]
    
    return r

records = list(map(lambda r: fix_record_list(r), records))

In [8]:
records = list(map(lambda r: dict(zip(columns, r)), records))
df_in = pd.DataFrame(records)
df_in = df_in.replace("N/A", np.NaN) # https://stackoverflow.com/a/34794112
df_in

,Procedure,Service,Description,HCPCS Code,Rev Code,MSDRG,NDC,Charges,Aetna HMO/POS/OA,Ambetter,...,Multiplan,Managed Medicaid - PeachState,Managed Medicare - Pruitt Health,Managed Medicare - Sonder,PHCS,Managed Medicare - Wellcare,United Healthcare,Self-Pay,Min Fee,Max Fee
0,MS579,"Inpatient,Other Skin",Subcutaneous Tissue And Breast Procedures Wit...,NaN,NaN,MS579,NaN,67369.46,33236.9,40150.93,...,53895.57,19320.99,22534.69,22534.69,47158.62,22534.69,41600.74,20210.84,19320.99,64240.25
1,MS653,Inpatient,Major Bladder Procedures With McC,NaN,NaN,MS653,NaN,149236.39,46440.6,69639.96,...,119896.96,34307.44,54042.29,54042.29,104909.84,54042.29,72214.3,44961.36,33634.74,123945.35
2,MS570,Inpatient,Skin Debridement With McC,NaN,NaN,MS570,NaN,104199.27,33236.9,36287.52,...,83359.42,10335.58,20547.69,20547.69,72939.49,20547.69,37590.01,31259.78,10335.58,83359.42
3,MS571,Inpatient,Skin Debridement With Cc,NaN,NaN,MS571,NaN,53634.25,22765,16571.82,...,42675.18,7680.62,12642.22,12642.22,37340.79,12642.22,21633.07,16003.19,7680.62,42675.18
4,MS572,Inpatient,Skin Debridement Without Cc/McC,NaN,NaN,MS572,NaN,44860.07,13659,11216.52,...,36100.18,5727.21,9162.32,9162.32,31587.66,9162.32,14609,13537.57,5727.21,36100.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55776,129456136,Supplies,AUG TIBIAL 10MM MBT 4 KNEE REVISION STEP WEDGE...,C1776,278,NaN,NaN,6625.5,4902.87,0,...,5631.68,642.67,0,0,4637.85,0,0,1987.65,0,5631.68
55777,129456126,Supplies,AUG TIBIAL 10MM MBT 2.5 KNEE REVISION STEP WED...,C1776,278,NaN,NaN,6625.5,4902.87,0,...,5631.68,642.67,0,0,4637.85,0,0,1987.65,0,5631.68
55778,129456125,Supplies,WEDGE STEP REV CEMENT 2.5X5MM,C1776,278,NaN,NaN,5962.11,4411.96,0,...,5067.8,578.32,0,0,4173.48,0,0,1788.63,0,5067.8
55779,129456121,Supplies,AUG TIBIAL 10MM MBT 2 KNEE REVISION STEP WEDGE...,C1776,278,NaN,NaN,6625.5,4902.87,0,...,5631.68,642.67,0,0,4637.85,0,0,1987.65,0,5631.68


In [9]:
df_mid = pd.DataFrame(df_in)

df_mid = df_mid.rename(columns={
    'Procedure': 'local_code',
    'Description': 'description',
    'HCPCS Code': 'hcpcs_cpt',
    'MSDRG': 'ms_drg',
    'Rev Code': 'rev_code',
    'NDC': 'ndc',
    'Service': 'setting',
})

df_mid

,local_code,setting,description,hcpcs_cpt,rev_code,ms_drg,ndc,Charges,Aetna HMO/POS/OA,Ambetter,...,Multiplan,Managed Medicaid - PeachState,Managed Medicare - Pruitt Health,Managed Medicare - Sonder,PHCS,Managed Medicare - Wellcare,United Healthcare,Self-Pay,Min Fee,Max Fee
0,MS579,"Inpatient,Other Skin",Subcutaneous Tissue And Breast Procedures Wit...,NaN,NaN,MS579,NaN,67369.46,33236.9,40150.93,...,53895.57,19320.99,22534.69,22534.69,47158.62,22534.69,41600.74,20210.84,19320.99,64240.25
1,MS653,Inpatient,Major Bladder Procedures With McC,NaN,NaN,MS653,NaN,149236.39,46440.6,69639.96,...,119896.96,34307.44,54042.29,54042.29,104909.84,54042.29,72214.3,44961.36,33634.74,123945.35
2,MS570,Inpatient,Skin Debridement With McC,NaN,NaN,MS570,NaN,104199.27,33236.9,36287.52,...,83359.42,10335.58,20547.69,20547.69,72939.49,20547.69,37590.01,31259.78,10335.58,83359.42
3,MS571,Inpatient,Skin Debridement With Cc,NaN,NaN,MS571,NaN,53634.25,22765,16571.82,...,42675.18,7680.62,12642.22,12642.22,37340.79,12642.22,21633.07,16003.19,7680.62,42675.18
4,MS572,Inpatient,Skin Debridement Without Cc/McC,NaN,NaN,MS572,NaN,44860.07,13659,11216.52,...,36100.18,5727.21,9162.32,9162.32,31587.66,9162.32,14609,13537.57,5727.21,36100.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55776,129456136,Supplies,AUG TIBIAL 10MM MBT 4 KNEE REVISION STEP WEDGE...,C1776,278,NaN,NaN,6625.5,4902.87,0,...,5631.68,642.67,0,0,4637.85,0,0,1987.65,0,5631.68
55777,129456126,Supplies,AUG TIBIAL 10MM MBT 2.5 KNEE REVISION STEP WED...,C1776,278,NaN,NaN,6625.5,4902.87,0,...,5631.68,642.67,0,0,4637.85,0,0,1987.65,0,5631.68
55778,129456125,Supplies,WEDGE STEP REV CEMENT 2.5X5MM,C1776,278,NaN,NaN,5962.11,4411.96,0,...,5067.8,578.32,0,0,4173.48,0,0,1788.63,0,5067.8
55779,129456121,Supplies,AUG TIBIAL 10MM MBT 2 KNEE REVISION STEP WEDGE...,C1776,278,NaN,NaN,6625.5,4902.87,0,...,5631.68,642.67,0,0,4637.85,0,0,1987.65,0,5631.68


In [10]:
set(df_mid['setting'].to_list())

{'Inpatient',
 'Inpatient,Abortion With D&C',
 'Inpatient,Acute Myocardial Infarction',
 'Inpatient,Aftercare',
 'Inpatient,Alcohol',
 'Inpatient,Alcohol, Drug Abuse Or Dependence',
 'Inpatient,Amputation Of Lower Limb For Endocrine',
 'Inpatient,Breast Biopsy',
 'Inpatient,Cardiac Arrest',
 'Inpatient,Cardiac Defibrillator Implant With Cardiac Catheterization With Ami',
 'Inpatient,Cardiac Defibrillator Implant With Cardiac Catheterization Without Ami',
 'Inpatient,Circulatory Disorders Except Ami',
 'Inpatient,D&C, Conization',
 'Inpatient,Disorders Of Liver Except Malignancy',
 'Inpatient,Ear, Nose',
 'Inpatient,Ecmo Or Tracheostomy With Mv >96 Hours Or Principal Diagnosis Except Face',
 'Inpatient,Esophagitis',
 'Inpatient,Extreme Immaturity Or Respiratory Distress Syndrome',
 'Inpatient,Fracture, Sprain, Strain And Dislocation Except Femur, Hip',
 'Inpatient,Hand Or Wrist Procedures',
 'Inpatient,Infections',
 'Inpatient,Ischemic Stroke',
 'Inpatient,Limb Reattachment',
 'Inpatien

In [11]:
money_idx = df_mid.columns.to_list().index('Charges')
money_columns = df_mid.columns[money_idx:]
remaining_columns = df_mid.columns[:money_idx]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')
df_mid

,local_code,setting,description,hcpcs_cpt,rev_code,ms_drg,ndc,payer_name,standard_charge
0,MS579,"Inpatient,Other Skin",Subcutaneous Tissue And Breast Procedures Wit...,NaN,NaN,MS579,NaN,Charges,67369.46
1,MS653,Inpatient,Major Bladder Procedures With McC,NaN,NaN,MS653,NaN,Charges,149236.39
2,MS570,Inpatient,Skin Debridement With McC,NaN,NaN,MS570,NaN,Charges,104199.27
3,MS571,Inpatient,Skin Debridement With Cc,NaN,NaN,MS571,NaN,Charges,53634.25
4,MS572,Inpatient,Skin Debridement Without Cc/McC,NaN,NaN,MS572,NaN,Charges,44860.07
...,...,...,...,...,...,...,...,...,...
1450301,129456136,Supplies,AUG TIBIAL 10MM MBT 4 KNEE REVISION STEP WEDGE...,C1776,278,NaN,NaN,Max Fee,5631.68
1450302,129456126,Supplies,AUG TIBIAL 10MM MBT 2.5 KNEE REVISION STEP WED...,C1776,278,NaN,NaN,Max Fee,5631.68
1450303,129456125,Supplies,WEDGE STEP REV CEMENT 2.5X5MM,C1776,278,NaN,NaN,Max Fee,5067.8
1450304,129456121,Supplies,AUG TIBIAL 10MM MBT 2 KNEE REVISION STEP WEDGE...,C1776,278,NaN,NaN,Max Fee,5631.68


In [12]:
if 'setting' in df_mid.columns:
    df_mid['additional_generic_notes'] = df_mid['setting']
    df_mid['setting'] = None
    df_mid.loc[df_mid['additional_generic_notes'].str.startswith('Inpatient'), 'setting'] = 'inpatient'
    df_mid.loc[df_mid['additional_generic_notes'].str.startswith('Outpatient'), 'setting'] = 'outpatient'

In [13]:
set(df_mid['setting'].to_list())

{None, 'inpatient', 'outpatient'}

In [14]:
df_mid.loc[df_mid['ms_drg'].notnull(), 'ms_drg'] = df_mid[df_mid['ms_drg'].notnull()]['ms_drg'].str.replace('MS', '')

In [15]:
df_mid.loc[df_mid['rev_code'].notnull(), 'rev_code'] = df_mid[df_mid['rev_code'].notnull()]['rev_code'].str.zfill(4)

In [16]:
df_mid.loc[df_mid['hcpcs_cpt'].notnull(), 'hcpcs_cpt'] = df_mid[df_mid['hcpcs_cpt'].notnull()]['hcpcs_cpt'].str.upper()

In [17]:
def payer_category_from_payer_name(payer_name):
    if payer_name == "Charges":
        return "gross"
    elif payer_name == "Self-pay":
        return "cash"
    elif payer_name == "Min Fee":
        return "min"
    elif payer_name == "Max Fee":
        return "max"

    return "payer"

df_mid['payer_category'] = df_mid['payer_name'].apply(payer_category_from_payer_name)
df_mid

,local_code,setting,description,hcpcs_cpt,rev_code,ms_drg,ndc,payer_name,standard_charge,additional_generic_notes,payer_category
0,MS579,inpatient,Subcutaneous Tissue And Breast Procedures Wit...,NaN,NaN,579,NaN,Charges,67369.46,"Inpatient,Other Skin",gross
1,MS653,inpatient,Major Bladder Procedures With McC,NaN,NaN,653,NaN,Charges,149236.39,Inpatient,gross
2,MS570,inpatient,Skin Debridement With McC,NaN,NaN,570,NaN,Charges,104199.27,Inpatient,gross
3,MS571,inpatient,Skin Debridement With Cc,NaN,NaN,571,NaN,Charges,53634.25,Inpatient,gross
4,MS572,inpatient,Skin Debridement Without Cc/McC,NaN,NaN,572,NaN,Charges,44860.07,Inpatient,gross
...,...,...,...,...,...,...,...,...,...,...,...
1450301,129456136,None,AUG TIBIAL 10MM MBT 4 KNEE REVISION STEP WEDGE...,C1776,0278,NaN,NaN,Max Fee,5631.68,Supplies,max
1450302,129456126,None,AUG TIBIAL 10MM MBT 2.5 KNEE REVISION STEP WED...,C1776,0278,NaN,NaN,Max Fee,5631.68,Supplies,max
1450303,129456125,None,WEDGE STEP REV CEMENT 2.5X5MM,C1776,0278,NaN,NaN,Max Fee,5067.8,Supplies,max
1450304,129456121,None,AUG TIBIAL 10MM MBT 2 KNEE REVISION STEP WEDGE...,C1776,0278,NaN,NaN,Max Fee,5631.68,Supplies,max


In [18]:
df_mid.dropna(subset=['standard_charge'], inplace=True)

In [19]:
df_mid['hospital_id'] = ccn
df_mid['line_type'] = None
df_mid['code'] = None
df_mid['apr_drg'] = None
df_mid['eapg'] = None
df_mid['modifiers'] = None
df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['ndc'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['billing_class'] = None
if not 'setting' in df_mid.columns:
    df_mid['setting'] = None
df_mid['plan_name'] = None
df_mid['standard_charge_percent'] = None
df_mid['contracting_method'] = None
if not 'additional_generic_notes' in df_mid.columns:
    df_mid['additional_generic_notes'] = None
df_mid['additional_payer_specific_notes'] = None

df_mid

,local_code,setting,description,hcpcs_cpt,rev_code,ms_drg,ndc,payer_name,standard_charge,additional_generic_notes,...,icd,drug_hcpcs_multiplier,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,billing_class,plan_name,standard_charge_percent,contracting_method,additional_payer_specific_notes
0,MS579,inpatient,Subcutaneous Tissue And Breast Procedures Wit...,NaN,NaN,579,None,Charges,67369.46,"Inpatient,Other Skin",...,None,None,None,None,None,None,None,None,None,None
1,MS653,inpatient,Major Bladder Procedures With McC,NaN,NaN,653,None,Charges,149236.39,Inpatient,...,None,None,None,None,None,None,None,None,None,None
2,MS570,inpatient,Skin Debridement With McC,NaN,NaN,570,None,Charges,104199.27,Inpatient,...,None,None,None,None,None,None,None,None,None,None
3,MS571,inpatient,Skin Debridement With Cc,NaN,NaN,571,None,Charges,53634.25,Inpatient,...,None,None,None,None,None,None,None,None,None,None
4,MS572,inpatient,Skin Debridement Without Cc/McC,NaN,NaN,572,None,Charges,44860.07,Inpatient,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450301,129456136,None,AUG TIBIAL 10MM MBT 4 KNEE REVISION STEP WEDGE...,C1776,0278,NaN,None,Max Fee,5631.68,Supplies,...,None,None,None,None,None,None,None,None,None,None
1450302,129456126,None,AUG TIBIAL 10MM MBT 2.5 KNEE REVISION STEP WED...,C1776,0278,NaN,None,Max Fee,5631.68,Supplies,...,None,None,None,None,None,None,None,None,None,None
1450303,129456125,None,WEDGE STEP REV CEMENT 2.5X5MM,C1776,0278,NaN,None,Max Fee,5067.8,Supplies,...,None,None,None,None,None,None,None,None,None,None
1450304,129456121,None,AUG TIBIAL 10MM MBT 2 KNEE REVISION STEP WEDGE...,C1776,0278,NaN,None,Max Fee,5631.68,Supplies,...,None,None,None,None,None,None,None,None,None,None


In [20]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,110229,None,Subcutaneous Tissue And Breast Procedures Wit...,NaN,MS579,None,579,None,None,NaN,...,None,inpatient,gross,Charges,None,67369.46,None,None,"Inpatient,Other Skin",None
1,110229,None,Major Bladder Procedures With McC,NaN,MS653,None,653,None,None,NaN,...,None,inpatient,gross,Charges,None,149236.39,None,None,Inpatient,None
2,110229,None,Skin Debridement With McC,NaN,MS570,None,570,None,None,NaN,...,None,inpatient,gross,Charges,None,104199.27,None,None,Inpatient,None
3,110229,None,Skin Debridement With Cc,NaN,MS571,None,571,None,None,NaN,...,None,inpatient,gross,Charges,None,53634.25,None,None,Inpatient,None
4,110229,None,Skin Debridement Without Cc/McC,NaN,MS572,None,572,None,None,NaN,...,None,inpatient,gross,Charges,None,44860.07,None,None,Inpatient,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450301,110229,None,AUG TIBIAL 10MM MBT 4 KNEE REVISION STEP WEDGE...,0278,129456136,None,NaN,None,None,C1776,...,None,None,max,Max Fee,None,5631.68,None,None,Supplies,None
1450302,110229,None,AUG TIBIAL 10MM MBT 2.5 KNEE REVISION STEP WED...,0278,129456126,None,NaN,None,None,C1776,...,None,None,max,Max Fee,None,5631.68,None,None,Supplies,None
1450303,110229,None,WEDGE STEP REV CEMENT 2.5X5MM,0278,129456125,None,NaN,None,None,C1776,...,None,None,max,Max Fee,None,5067.8,None,None,Supplies,None
1450304,110229,None,AUG TIBIAL 10MM MBT 2 KNEE REVISION STEP WEDGE...,0278,129456121,None,NaN,None,None,C1776,...,None,None,max,Max Fee,None,5631.68,None,None,Supplies,None


In [21]:
df_out.to_csv("rate_" + ccn + ".csv", index=False)

In [22]:
from dateutil.parser import parse as parse_datetime

df_tmp = pd.read_excel(filename)
last_updated = df_tmp.iloc[1][0]
last_updated = last_updated.split(' ')[-1]
last_updated = parse_datetime(last_updated).isoformat().split("T")[0]
last_updated

'2022-01-01'